In [1]:
import geopandas as gpd
from geopandas.geoseries import *

# 447 PLRs
file_path_of_plr = 'berlin-lor.planungsraeume.geojson'
plr = gpd.read_file(file_path_of_plr)
projection = "EPSG:5678" # https://epsg.io/5243 , https://epsg.io/5678 , https://epsg.io/3068
bufferSize = 300 # in meter
plr = plr.to_crs(projection)
 
# calc area (for fun ;-)) 
plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None

plr['bufferedGeometry'] = plr.geometry.buffer(bufferSize)
plr['bufferedArea'] = None

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = row['geometry'].area / 10000
    plr.loc[index, 'bufferedArea'] = row['bufferedGeometry'].area / 10000

# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen_shapes.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)
# 2509 open spaces
open_spaces = open_spaces.to_crs(projection)

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()

plr['allOpenSpacesArea'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesBufferedArea'] = np.empty((len(plr), 0)).tolist()

for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(rowOS.KENNZEICH)
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowOS.KATASTERFL)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(rowOS.KENNZEICH)
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowOS.KATASTERFL)
        if rowPLR.SCHLUESSEL.startswith('0'):
            plr.loc[indexPLR, 'SCHLUESSEL'] = rowPLR.SCHLUESSEL[1:]

newplr = plr
newplr.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('PLR_buffered.csv', sep=';')


In [2]:
newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)

import pandas as pd 
data = pd.read_csv('EWR201812E_Matrix.csv', sep=';')
data.rename(columns = {'RAUMID':'SCHLUESSEL'}, inplace = True)
data = data[['SCHLUESSEL', 'E_E']]

data['SCHLUESSEL']=data['SCHLUESSEL'].astype(str)
newplr['SCHLUESSEL']=newplr['SCHLUESSEL'].astype(str)

output = pd.merge(newplr, data, on='SCHLUESSEL')
output = output.rename(columns={'E_E': 'population'})

output['openSpacesPerInhabitant'] = None

for index, row in output.iterrows():
    output.loc[index, 'allOpenSpacesBufferedArea'] = sum(row.allOpenSpacesBufferedArea) / 10000
    output.loc[index, 'allOpenSpacesArea'] = sum(row.allOpenSpacesArea) / 10000
    output.loc[index, 'openSpacesPerInhabitant'] = output.loc[index, 'allOpenSpacesBufferedArea'] * 10000 / row.population    

output.head()

output.to_csv('data.csv', index = False, header=True)

# TODOs
# Als Einwohner von PLR x hat man y Grünflächen mit z m^2 Fläche zur Verfügung 
# check Einheiten
# Daten für Waldflächen

,SCHLUESSEL,PLR_NAME,geometry,area,bufferedGeometry,bufferedArea,openSpacesInBufferedPLR,openSpacesInPLR,allOpenSpacesArea,allOpenSpacesBufferedArea,numberOfSpacesInPLR,numberOfSpacesInBufferedPLR,population,openSpacesPerInhabitant
0,1011101,Stülerstr.,"POLYGON ((4591014.544 5819985.375, 4591080.254...",36.7055,"POLYGON ((4590729.984 5819890.373, 4590722.091...",150.52,"[216210, 400000, 217100, 242700, 216810, 21670...","[400000, 242700, 212610]",200.011,201.732,3,7,3384,596.134
1,1011102,Großer Tiergarten,"POLYGON ((4590490.781 5820805.462, 4590491.763...",301.156,"POLYGON ((4590161.505 5820667.145, 4590158.549...",608.388,"[253500, 244010, 253400, 161900, 210100, 21269...","[253500, 244010, 210100, 212690, 400000, 24270...",222.99,242.823,21,50,196,12388.9
2,1011103,Lützowstr.,"POLYGON ((4591721.796 5819736.719, 4591754.196...",52.2729,"POLYGON ((4591533.106 5819979.445, 4591539.149...",176.848,"[216310, 216500, 216210, 206110, 216400, 21710...","[216500, 216210, 216400]",1.96,6.96893,3,9,5038,13.8327
3,1011104,Körnerstr.,"POLYGON ((4592632.388 5819374.203, 4592639.827...",33.8588,"POLYGON ((4592545.285 5819087.126, 4592517.674...",132.499,"[253310, 50224, 50004, 63361, 216310, 216500, ...","[63361, 216310]",7.5689,21.8316,2,18,4759,45.8744
4,1011105,Nördl. Landwehrkanal,"POLYGON ((4591867.807 5820484.277, 4591895.354...",93.5224,"POLYGON ((4591590.290 5820370.327, 4591580.473...",272.056,"[253310, 50224, 50004, 50087, 50015, 253400, 2...","[253310, 253400, 211500, 206110, 201900, 20613...",172.142,186.065,11,22,1421,1309.4
